In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import polars as pl
import requests

In [13]:
predictions_url = 'https://api-v3.mbta.com/prediction'
with open('api_key.txt') as file:
    api_key = file.read()

In [27]:
routes = requests.get('https://api-v3.mbta.com/routes').json()
for route in routes['data']:
    route_id = route['id']
    print(f'Stop ID: {route_id}')

Stop ID: Red
Stop ID: Mattapan
Stop ID: Orange
Stop ID: Green-B
Stop ID: Green-C
Stop ID: Green-D
Stop ID: Green-E
Stop ID: Blue
Stop ID: 741
Stop ID: 742
Stop ID: 743
Stop ID: 751
Stop ID: 749
Stop ID: 746
Stop ID: CR-Fairmount
Stop ID: CR-Fitchburg
Stop ID: CR-Worcester
Stop ID: CR-Franklin
Stop ID: CR-Greenbush
Stop ID: CR-Haverhill
Stop ID: CR-Kingston
Stop ID: CR-Lowell
Stop ID: CR-Middleborough
Stop ID: CR-Needham
Stop ID: CR-Newburyport
Stop ID: CR-Providence
Stop ID: CR-Foxboro
Stop ID: Boat-F4
Stop ID: Boat-F1
Stop ID: Boat-EastBoston
Stop ID: Boat-Lynn
Stop ID: Boat-F6
Stop ID: 747
Stop ID: 708
Stop ID: 1
Stop ID: 4
Stop ID: 7
Stop ID: 8
Stop ID: 9
Stop ID: 10
Stop ID: 11
Stop ID: 14
Stop ID: 15
Stop ID: 16
Stop ID: 17
Stop ID: 18
Stop ID: 19
Stop ID: 21
Stop ID: 22
Stop ID: 23
Stop ID: 24
Stop ID: 26
Stop ID: 28
Stop ID: 29
Stop ID: 30
Stop ID: 31
Stop ID: 32
Stop ID: 33
Stop ID: 34
Stop ID: 34E
Stop ID: 35
Stop ID: 36
Stop ID: 37
Stop ID: 38
Stop ID: 39
Stop ID: 40
Stop ID:

In [28]:
stops = requests.get('https://api-v3.mbta.com/stops', params={'filter[route]': 'Green-D'}).json()
for stop in stops['data']:
    stop_id = stop['id']
    stop_name = stop['attributes']['name']
    print(f'Stop ID: {stop_id}, Stop Name: {stop_name}')

Stop ID: place-river, Stop Name: Riverside
Stop ID: place-woodl, Stop Name: Woodland
Stop ID: place-waban, Stop Name: Waban
Stop ID: place-eliot, Stop Name: Eliot
Stop ID: place-newtn, Stop Name: Newton Highlands
Stop ID: place-newto, Stop Name: Newton Centre
Stop ID: place-chhil, Stop Name: Chestnut Hill
Stop ID: place-rsmnl, Stop Name: Reservoir
Stop ID: place-bcnfd, Stop Name: Beaconsfield
Stop ID: place-brkhl, Stop Name: Brookline Hills
Stop ID: place-bvmnl, Stop Name: Brookline Village
Stop ID: place-longw, Stop Name: Longwood
Stop ID: place-fenwy, Stop Name: Fenway
Stop ID: place-kencl, Stop Name: Kenmore
Stop ID: place-hymnl, Stop Name: Hynes Convention Center
Stop ID: place-coecl, Stop Name: Copley
Stop ID: place-armnl, Stop Name: Arlington
Stop ID: place-boyls, Stop Name: Boylston
Stop ID: place-pktrm, Stop Name: Park Street
Stop ID: place-gover, Stop Name: Government Center
Stop ID: place-haecl, Stop Name: Haymarket
Stop ID: place-north, Stop Name: North Station
Stop ID: plac

In [29]:
params1 = {'api_key': api_key,
          'filter[stop]': 'place-gover'}

In [35]:
response1 = requests.get(f'https://api-v3.mbta.com/predictions?filter[stop]=place-gover&api_key={api_key}').json()

In [45]:
response1['data'][0]

{'attributes': {'arrival_time': None,
  'arrival_uncertainty': None,
  'departure_time': '2024-05-19T23:52:45-04:00',
  'departure_uncertainty': 360,
  'direction_id': 0,
  'last_trip': False,
  'revenue': 'REVENUE',
  'schedule_relationship': 'ADDED',
  'status': None,
  'stop_sequence': 40},
 'id': 'prediction-ADDED-1581911196-70202-40',
 'relationships': {'route': {'data': {'id': 'Green-C', 'type': 'route'}},
  'stop': {'data': {'id': '70202', 'type': 'stop'}},
  'trip': {'data': {'id': 'ADDED-1581911196', 'type': 'trip'}},
  'vehicle': {'data': {'id': 'G-10301', 'type': 'vehicle'}}},
 'type': 'prediction'}